In [1]:
!pip install transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=c2b05ed68ecc981132133242568c77aca904a264f7e33dbdbdccfb1bdc31619b
  Stored in directory: /root/.cache/pip/wheels

In [2]:
import wandb
wandb.login(key="aec6fef7ba56ee445129472eb583718b8e529934")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
import pickle
import os
import copy
import sys
from tqdm import tqdm
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score, recall_score, roc_curve, auc

In [4]:
def deflog(text):
    print(f"[DEF-LOG] {text}")

In [8]:
# %cd /content/drive/MyDrive/mlops/mlops_part_2_progect/reviews
!cp -r "/content/drive/MyDrive/Archive" "/content"

In [6]:
config = {
  "lr": 1e-3,
  "weight_decay": 1e-4,
  "epoch_num": 100,
  "batch_size": 128
}
tags = ["new_data", "10percent", "BERT-Pretrained"]
run = wandb.init(entity='sentimovie', project="GGGGGGGOOOOVVVVVV", config=config, tags=tags)

wandb: Currently logged in as: hamid-mj3 (team_h). Use `wandb login --relogin` to force relogin


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

In [10]:
# from transformers import AutoModel, BertTokenizerFast
# #load model and tokenizer
# PRE_TRAINED_MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"
# roberta = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
# tokenizer = BertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
with open('Archive/train.pkl', 'rb') as f:
    train_set = pickle.load(f)
    
with open('Archive/test.pkl', 'rb') as f:
    test_set = pickle.load(f)

with open('Archive/val.pkl', 'rb') as f:
    val_set = pickle.load(f)

df_train = pd.DataFrame.from_dict(train_set)
df_test =  pd.DataFrame.from_dict(test_set)
df_val =  pd.DataFrame.from_dict(val_set)
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()

data = {'train': {}, "test": {}, 'val': {}}
data['train']['x'] = list(df_train['x'])[:1000]
data['train']['y'] = list(df_train['y'])[:1000]
data['test']['x'] = list(df_test['x'])[:200]
data['test']['y'] = list(df_test['y'])[:200]
data['val']['x'] = list(df_val['x'])[:200]
data['val']['y'] = list(df_val['y'])[:200]

del df_train
del df_test
del df_val
del train_set
del test_set
del val_set

In [17]:
# class BERT_architecture(nn.Module):
#     def __init__(self, bert):
#         super(BERT_architecture, self).__init__()
#         self.bert = bert
#         for param in bert.parameters():
#             param.requires_grad = False
#         self.fc = nn.Linear(768,2)
    
#     def forward(self, token_ids):
#         _, cls_hs = self.bert(token_ids, return_dict=False)
#         x = self.fc(cls_hs)
#         return x

from transformers import DistilBertTokenizer, AutoModelForSequenceClassification, AutoTokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

for name, param in model.named_parameters():
    if name.startswith('roberta'):
        param.requires_grad = False


    # param.requires_grad = False
# i = 0
# for param in bert.parameters():
#     if i > 195:
#         param.requires_grad = True  
#     i += 1
# model = bert
# model

NameError: ignored

In [13]:
# model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [18]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


class MyDataset(Dataset):
    def __init__(self, X_data, labels):
        self.x_data = X_data
        self.labels = np.unique(labels, return_inverse=True)[1]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label_onehot = torch.tensor(self.labels[idx])
        embedding = tokenizer(self.x_data[idx], return_tensors='pt', 
                              padding="max_length", max_length=170, truncation=True)['input_ids']
        embedding = embedding.squeeze()
        return embedding, label_onehot


def dataset_loader(data, config):
    dataset = {'train': MyDataset(data['train']['x'], data['train']['y']),
              'val': MyDataset(data['val']['x'], data['val']['y']),
              'test': MyDataset(data['test']['x'], data['test']['y'])}
    deflog("Datasets created")

    dataloaders = {x: DataLoader(dataset[x], batch_size=config['batch_size'], 
                                 num_workers=1) for x in ['train', 'val', 'test']}
    deflog("Dataloaders created")
    dataset_sizes = {x: len(dataset[x]) for x in ['train', 'val', 'test']}
    deflog("Dataset_sizes created")
    return dataloaders, dataset, dataset_sizes

In [28]:
def train_model(model, criterion, optimizer, dataloaders, 
                dataset_sizes, device, config, earlyStopper, num_epochs=25):
    since = time.time()
    history = {'train_loss': [], 'val_loss': [], 
                    'train_acc': [], 'val_acc': []}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in tqdm(range(num_epochs)):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    _, outputs = model(inputs, return_dict=False)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, outputs)
                    # outputs = model(inputs)
                    # _, preds = torch.max(outputs, 1)
                    # loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            # if phase == 'train':
            #     scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(float(epoch_acc.cpu().numpy()))

            # deflog(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            #######################################
            ###           wandb logging         ###
            #######################################
            ##        Log the training flow       ##
            #######################################
            tmp = {f'{phase}_loss': epoch_loss, 
                    f'{phase}_acc': epoch_acc,
                    'lr': config['lr'],
                    'epoch': epoch}
            wandb.log(tmp)
            #######################################
            #######################################

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        if earlyStopper.early_stop(history['val_loss'][-1]):
            deflog("Early Stopping")
            break

    time_elapsed = time.time() - since
    deflog(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    deflog(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

In [29]:
# dataset and dataloader
dataloaders, dataset, dataset_sizes = dataset_loader(data, config)

# model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
deflog(f"ML model defined (device: {device})")

earlyStopper = EarlyStopper(patience=10, min_delta=1e-2)
deflog("EarlyStopper defined")

class_weights = compute_class_weight('balanced', 
                                    classes=np.unique(data['train']['y']), 
                                    y=np.array(data['train']['y']))
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
# criterion = nn.CrossEntropyLoss()
deflog("Define loss class_weights")

optimizer = optim.AdamW(model.parameters(), lr=wandb.config['lr'],
                        weight_decay=wandb.config['weight_decay'])
deflog("Optimizer (Adam) defined")

cosin_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 
                                                    T_max=config['epoch_num'], 
                                                    eta_min=5e-6)
deflog("Scheduler (cosin_lr_scheduler) defined")

model, history = train_model(model, criterion, optimizer, 
                              dataloaders, dataset_sizes, device, config,
                              earlyStopper, config['epoch_num'])

[DEF-LOG] Datasets created
[DEF-LOG] Dataloaders created
[DEF-LOG] Dataset_sizes created
[DEF-LOG] ML model defined (device: cuda:0)
[DEF-LOG] EarlyStopper defined
[DEF-LOG] Define loss class_weights
[DEF-LOG] Optimizer (Adam) defined
[DEF-LOG] Scheduler (cosin_lr_scheduler) defined


  0%|          | 0/100 [00:00<?, ?it/s]


ValueError: ignored